In [1]:
import cv2
import numpy as np

In [2]:
def show(img):
    cv2.imshow('img', img)
    cv2.waitKey(0)

def play(vid, func = None):
    while True:
        success, frame = vid.read()
        if func:
            frame = func(frame)
        cv2.imshow('frame', frame)
        if cv2.waitKey(16) & 0XFF==ord('q'):
            break
    cv2.destroyAllWindows()

### Chapter 1

In [6]:
def gray(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
play(cap, gray)

In [9]:
lenna = cv2.imread('resources/lenna forsen.jpg')
show(lenna)

### Chapter 2

In [47]:
# cap = cv2.VideoCapture('resources/video.mp4')
cap = cv2.VideoCapture(1)

play(cap)

In [24]:
def canny(img):
    return cv2.Canny(img, 144, 144)
play(cap, canny)

In [23]:
def dilate(img):
    return cv2.dilate(canny(img), (3, 3), iterations = 1)
play(cap, dilate)

In [27]:
def erode(img):
    return cv2.erode(canny(img), (3, 3), iterations = 1)
play(cap, erode)

### Chapter 3

In [26]:
def resize(img):
    return cv2.resize(img,( 256, 144))
play(cap, resize)

In [20]:
def crop(img, x=300,y=100,h=300,w=300):
    return img[y:y+w, x:x+h]
play(cap, crop)

### Chapter 4

cv2.line, cv2.rectangle, cv2.circle, cv2.putText

### Chapter 5

we can change the perspective to transform:
matrix = cv2.getPerspectiveTransform(
    points in original picture,
    points in new picture ex: np.float32([[0, 0], [w, 0], [0, h], [w, h]]), 
)
imOutput = cv2.warpPerspective(img, matrix, (w, h))

### Chapter 6

In [22]:
def qstack(img):
    return np.hstack((np.vstack((
        crop(img), crop(img))), np.vstack((
        crop(img), crop(img)))))
play(cap, qstack)

### Chapter 7

In [15]:
def hsv(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
play(cap, hsv)

In [21]:
def control(img):
    def empty(a):
        pass
    cv2.namedWindow('TrackBar')
    cv2.resizeWindow('TrackBar', 640, 240)
    cv2.createTrackbar('hue Min', 'TrackBar', 0, 179, empty)
    cv2.createTrackbar('hue Max', 'TrackBar', 179, 179, empty)
    cv2.createTrackbar('sat Min', 'TrackBar', 0, 255, empty)
    cv2.createTrackbar('sat Max', 'TrackBar', 255, 255, empty)
    cv2.createTrackbar('val Min', 'TrackBar', 0, 255, empty)
    cv2.createTrackbar('val Max', 'TrackBar', 255, 255, empty)

    while True:
        h_min = cv2.getTrackbarPos('hue Min', 'TrackBar')
        h_max = cv2.getTrackbarPos('hue Max', 'TrackBar')
        s_min = cv2.getTrackbarPos('sat Min', 'TrackBar')
        s_max = cv2.getTrackbarPos('sat Max', 'TrackBar')
        v_min = cv2.getTrackbarPos('val Min', 'TrackBar')
        v_max = cv2.getTrackbarPos('val Max', 'TrackBar')

        lower = np.array([h_min, s_min, v_min])
        upper = np.array([h_max, s_max, v_max])
        
        mask = cv2.inRange(hsv(img), lower, upper)
        cv2.imshow('mask', mask)
        cv2.imshow('masked', cv2.bitwise_and(img, img, mask=mask))
        
        
#         if cv2.waitKey(1) or 0XFF==ord('q'):
#             break
        cv2.waitKey(1)

control(lenna)        

KeyboardInterrupt: 

### Chapter 8

In [42]:
def getContours(img):
    contours, hierarchy = cv2.findContours(canny(img), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    t = img.copy()
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 500: 
            continue
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
        x,y,w,h = cv2.boundingRect(approx)
        cv2.rectangle(t, (x,y), (x+w,y+h), (0,255,0), 2)
        cv2.putText(t, str(len(approx)), (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,255), 2)

#     cv2.drawContours(t, contours, -1, (0,255,0), 2)
    return t
    
play(cap, getContours)

### Chapter 9

In [44]:
faceDetector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [45]:
def detectFace(img):
    g = gray(img)
    faces = faceDetector.detectMultiScale(g, 1.1, 4)
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
    return img
play(cap, detectFace)

In [46]:
upperDetector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_upperbody.xml')
def detectupper(img, cascade=upperDetector):
    g = gray(img)
    rects = cascade.detectMultiScale(g, 1.1, 4)
    for (x,y,w,h) in rects:
        cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
    return img
play(cap, detectupper)